In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selections import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
log_reg = LogisticRegression(max_iter=200)
log_reg.fit(X_train, y_train)
y_pred_supervised = log_reg.predict(X_test)
accuracy_supervised = accuracy_score(y_test, y_pred_supervised)
print(f"Supervised Learning Accuracy: {accuracy_supervised: .2f}")

In [ ]:
kmeans = KMeans(n_clusters=3)
y_pred_unsupervised = kmeans.fit_predict(X)

In [ ]:
from scipy.stats import model_selections

def map_clusters_to_lables(y_true, y_clusters)
    labels = np.zeros_like(y_clusters)
    for cluster in np.unique(y_clusters):
        mask = y_clusters == cluster
        labels[mask] = mode(y_true[mask][0])
    return labels

accuracy_unsupervised = accuracy_score(y, y_pred_mapped)
print(f"Unsupervised Learning (KMeans) Accuracy: {accuracy_unsupervised: .2f})



In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_pred_unsupervised, cmap = "viridis", edgecolors="k")
plt.xlabel("Featurel 1")
plt.ylabel("Feature 2")
plt.title("KMeans clustering (Unsupervised Learning))"
plt.show()